In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import cross_val_score, GridSearchCV

%matplotlib inline

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

Если нужно запустить ноутбук, то можно раскомментировать закомментированные строчки, чтобы получить файлы `*.compacted.csv`, которые будут нужны для обучения, а можно (и нужно, так как закомментированный код работает очень долго) скачать их по [ссылке](https://yadi.sk/d/qiGx18123TjDF6).

### Preprocessing

Для событий была предпринята попытка посчитать их количество для каждой команды для каждого `mid`. Так как это считается довольно медленно, это было посчитано один раз и сохранено в `actions.compacted.csv`

In [3]:
events = pd.read_csv('events.csv')
def count_events(event_type, from_team):
    def get_series(mid):
        return len(events[(events.mid == mid) & (events.event_type == event_type) & (events.from_team == from_team)])
    
    return get_series

actions = ['taken_aegis', 'stolen_aegis', 'enemy_barracks_destroyed', 'first_blood',
           'killed_roshan', 'self_tower_destroyed', 'enemy_tower_destroyed']
teams = ['radiant', 'dire']
# df = pd.DataFrame(data={'mid': list(range(49948))})
# for i, action in enumerate(actions):
#     for team in teams:
#         print(f'{team}_{action}')
#         df[f'{team}_{action}'] = df['mid'].map(count_events(i+4, team))
#     df.to_csv('actions.compacted.csv', index=False)

Также для некоторых событий была предпринята попытка посчитать время до первого такого события для команды и добавить в признаки разность времени для `radiant` и `dire`. Например, были добавлены признаки `time_to_first_tower_diff` и `time_to_first_blood_diff`. Однако, эти признаки не улучшили модель.

In [4]:
def get_time_diff(event_type):
    def f(mid):
        table = e[(e.mid == mid) & (e.event_type == event_type)]
        radiant_time = table[table.from_team == 'radiant']['time']
        dire_time = table[table.from_team == 'dire']['time']
        radiant_time = 0 if len(radiant_time) == 0 else radiant_time.iloc[0]
        dire_time = 0 if len(dire_time) == 0 else dire_time.iloc[0]
        
        return radiant_time - dire_time
    return f

# train['time_to_first_tower_diff'] = train['mid'].map(get_time_diff(6))
# train['time_to_first_blood_diff'] = train['mid'].map(get_time_diff(3))

Также было посчитано количество каждого из предметов для команд. Эти вычисления тоже заняли продолжительное время, поэтому результат был сохранен в `items.compacted.csv`.

In [5]:
items = pd.read_csv('items.csv')
def get_item(mid, item, radiant):
    lower = 0 if radiant else 5
    upper = 4 if radiant else 9
    return items[(items.mid == mid) & (items.player >= lower) & (items.player <= upper)][f'item_{item}'].sum()

# df = pd.DataFrame(data={'mid': list(range(49948))})
# for i in range(0, 121):
#     print(f'item={i}')
#     df[f'item_{i}_radiant'] = df['mid'].map(lambda mid: get_item(mid, i, True))
#     df[f'item_{i}_dire'] = df['mid'].map(lambda mid: get_item(mid, i, False))
#     df.to_csv(f'items.compacted.csv', index=False)

Для того, чтобы отобрать полезные для предсказания предметы, по всем предметам были построены графики зависимости `radiant_won` от разности между количеством определенного предмета у команд. В дальнейшем это не пригодилось, потому что результат логистической регрессии оказался лучше при обучении на разности для всех 120 предметов.

In [6]:
def hist_by_radiant_win(df, field, bins=50):
    groups = df.groupby('radiant_won')[field]
    fig, ax = plt.subplots()
    for k, v in reversed(list(groups)):
        v.hist(label=str(k), alpha=.75, ax=ax, bins=bins)

    ax.legend(title='radiant_won')

In [7]:
df = pd.read_csv('items.compacted.csv')
def calculate_diff_item(item):
    def get_value(mid):
        row = df[df.mid == mid]
        val = row[f'item_{item}_radiant'] - row[f'item_{item}_dire']
        if mid % 10000 == 0:
            print(mid)
        return val.iloc[0]
    return get_value

In [8]:
useful_items = [15, 19, 24, 26, 27, 28, 32,
                33, 43, 45, 53, 56, 66, 98, 112]

Для таблиц `gold`, `lh`, `xp` были сагрегированы данные по командам. Было посчитано суммарное значение для команды за 5 и 10 минут, а также суммарное значение для 4 наиболее сильных игроков в команде. Результирующие таблицы были также сохранены в `gold.compacted.csv`, `lh.compacted.csv`, `xp.compacted.csv`.

In [9]:
def compact(name):
    table = pd.read_csv(f'{name}.csv')
    
    radiant = [f'player_{i}' for i in range(5)]
    dire = [f'player_{i}' for i in range(5, 10)]
    
    table[f'{name}_radiant'] = table[radiant].sum(axis=1)
    table[f'{name}_dire'] = table[dire].sum(axis=1)
    
    table[f'{name}_radiant_5min'] = table[f'{name}_radiant'].shift(5)
    table[f'{name}_dire_5min'] = table[f'{name}_dire'].shift(5)
    
    table[f'{name}_radiant_4'] = table[radiant].sum(axis=1) - table[radiant].min(axis=1) + 1
    table[f'{name}_dire_4'] = table[dire].sum(axis=1) - table[dire].min(axis=1) + 1
    
    table = table[table.times == 600].drop(['times'] + radiant + dire, axis=1)
    
    table.to_csv(f'{name}.compacted.csv', index=False)
    return table

# compact('gold')
# compact('lh')
# compact('xp')

Далее создается мешок слов по героям. Для каждого `mid` для каждого героя ставится 1, если герой был в игре и играл за `radiant`, -1, если герой был и играл за `dire` и 0, если героя не было в игре. Результат сохраняется в `heroes.compacted.csv`.

In [10]:
def get_heroes():
    N = 110
    heroes = np.zeros((data.shape[0], N))

    for i, match_id in enumerate(data.index):
        for p in range(5):
            heroes[i, data.ix[match_id, f'player_{p}']-1] = 1
        for p in range(5, 10):
            heroes[i, data.ix[match_id, f'player_{p}']-1] = -1

    return heroes

# heroes = pd.DataFrame({'mid': list(range(49948))})
# heroes = heroes.join(pd.DataFrame(get_heroes(), columns=[f'hero_{i}' for i in range(110)]))
# heroes.to_csv('heroes.compacted.csv', index=False)

### Feature extraction

При генерировании признаков было написано два метода: `enrich` и `enrich_diff`. В `enrich` отдельно записывались признаки для `radiant` и `dire`. В `enrich_diff` для всех таких признаков взята разница `feature_radiant` - `feature_dire`. Модель, создаваемая в `enrich_diff` оказалась лучше, поэтому в дальнейшем используется она.

In [11]:
events = pd.read_csv('actions.compacted.csv')
items = pd.read_csv('items.compacted.csv')
gold = pd.read_csv('gold.compacted.csv')
lh = pd.read_csv('lh.compacted.csv')
xp = pd.read_csv('xp.compacted.csv')

useful_events = ['first_blood', 'killed_roshan', 'enemy_barracks_destroyed', 'self_tower_destroyed', 'enemy_tower_destroyed']

def enrich(data):
    data = data.merge(events[['mid'] +
                             [f'radiant_{event}' for event in useful_events] +
                             [f'dire_{event}' for event in useful_events]], on='mid', how='left')

    data = data.merge(heroes, on='mid', how='left')
    
    data = data.merge(items[['mid'] +
                            [f'item_{x}_radiant' for x in range(121)] +
                            [f'item_{x}_dire' for x in range(121)]], on='mid', how='left')

    data = data.merge(gold, on='mid', how='left')
    data = data.merge(lh, on='mid', how='left')
    data = data.merge(xp, on='mid', how='left')
    
    return data

def merge(data, table, name, names):
    table[f'{name}_diff'] = table[f'{name}_radiant'] - table[f'{name}_dire']
    table[f'{name}_5min_diff'] = table[f'{name}_radiant_5min'] - table[f'{name}_dire_5min']
    table[f'{name}_4players_diff'] = table[f'{name}_radiant_4'] - table[f'{name}_dire_4']
    table[f'{name}_diff_sqrt'] = (table[f'{name}_radiant'] - table[f'{name}_dire']) / np.sqrt(table[f'{name}_radiant'] + table[f'{name}_dire'])
    table[f'{name}_ratio'] = table[f'{name}_radiant'] / table[f'{name}_dire']
    return data.merge(table[['mid'] + [f'{name}_{x}' for x in names]], on='mid', how='left')
    
def enrich_diff(data):
    for event in useful_events:
        events[f'{event}_diff'] = events[f'radiant_{event}'] - events[f'dire_{event}']
        
    data = data.merge(events[['mid'] + [f'{event}_diff' for event in useful_events]], on='mid', how='left')
    
    heroes = pd.read_csv('heroes.compacted.csv')
    data = data.merge(heroes, on='mid', how='left')
    
    for item in range(121):
        items[f'item_{item}_diff'] = items[f'item_{item}_radiant'] - items[f'item_{item}_dire']
        
    data = data.merge(items[['mid'] + [f'item_{item}_diff' for item in range(121)]], on='mid', how='left')
    
    data = merge(data, gold, 'gold', ['diff', '5min_diff', '4players_diff'])
    data = merge(data, lh, 'lh', ['diff', 'diff_sqrt'])
    data = merge(data, xp, 'xp', ['diff', 'diff_sqrt'])

    return data

### Обучение модели

При обучении сначала использовался случайный лес, однако, логистическая регрессия показала себя значительно лучше, поэтому в дальнейшем используется она. Для масштабированя признаков было опробовано не масштабировать признаки вообще, использовать `StandardScaler` или использовать `RobustScaler`. `StandardScaler` показал наилучший результат. При обучении линейной регрессии были испробованы регуляризация L1 и L2, L1 показала лучший результат. Также для подбора параметра регуляризации `C` был использован `GridSearchCV`, с помощью которого был найден оптимальный параметр `C=0.548`.

In [12]:
def save_result(clf, scale=False):
    X = train.drop('radiant_won', axis=1).drop('mid', axis=1)
    y = train['radiant_won']
    X_test = test.drop('mid', axis=1)
    if scale:
        scaler = StandardScaler()
        X[:] = scaler.fit_transform(X)
        X_test[:] = scaler.transform(X_test)
    clf.fit(X, y)
    pred = clf.predict_proba(X_test)[:,1:]
    pred_data = pd.read_csv('test.csv')
    pred_data['radiant_won'] = pred
    pred_data.to_csv('res.csv', index=False)

In [13]:
train = enrich_diff(pd.read_csv('train.csv'))
test = enrich_diff(pd.read_csv('test.csv'))

In [14]:
X = train.drop('radiant_won', axis=1).drop('mid', axis=1)
y = train['radiant_won']

In [15]:
scaler = StandardScaler()
X[:] = scaler.fit_transform(X)

In [16]:
param_grid = {'C': [0.54, 0.0548, 0.55]}
optimizer = GridSearchCV(LogisticRegression(penalty='l1', random_state=42),
                         param_grid, scoring='roc_auc', cv=10, verbose=5, n_jobs=-1)
optimizer.fit(X, y)
optimizer.best_estimator_, optimizer.best_score_

Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done  22 out of  30 | elapsed:   49.6s remaining:   18.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.0min finished


(LogisticRegression(C=0.0548, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l1', random_state=42,
           solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
 0.7676739313139288)

In [17]:
save_result(LogisticRegression(random_state=42, C=0.0548, penalty='l1'), scale=True)

---

Была также предпринята попытка добавить в признаки пары героев (предполагалось, что некоторые герои хорошо играют в паре), для этого посчитаны винрейты для пар героев. Однако, эти признаки не сказались положительно на качестве и в финальной модели не присутствовали.

In [18]:
train = pd.read_csv('train.csv')
heroes = pd.read_csv('heroes.compacted.csv')
cols = []
for i in range(110):
    for j in range(i + 1, 110):
        col = f'heroes_{i}_{j}'
        cols.append(col)
        heroes[col] = heroes[f'hero_{i}'] + heroes[f'hero_{j}']

train = train.merge(heroes[['mid'] + cols], on='mid', how='left')

win_rates = {}
for i in range(110):
    for j in range(i+1, 110):
        col = f'heroes_{i}_{j}'
        plays_count = len(train[(train[col] == -2) | (train[col] == 2)])
        win_rates[col] = 0 if plays_count == 0 else (
            len(train[(train[col] == -2) & (train['radiant_won'] == 0)]) +
            len(train[(train[col] == 2) & (train['radiant_won'] == 1)])
        ) / plays_count, plays_count

In [19]:
interesting_pairs = []
for k,(v,n) in win_rates.items():
    if (v < 0.35 and v != 0 or v > 0.65) and n > 150:
        print(k,v,n)
        interesting_pairs.append(k)

heroes_7_16 0.34806629834254144 181
heroes_19_64 0.6583850931677019 161
heroes_23_58 0.345 200
heroes_28_49 0.654054054054054 185
heroes_28_67 0.6511627906976745 172
heroes_28_88 0.6545454545454545 220
heroes_28_108 0.7401960784313726 204
heroes_31_49 0.6538461538461539 234
heroes_38_67 0.6547619047619048 168
heroes_39_88 0.6772727272727272 220
heroes_49_68 0.6625 160
heroes_56_64 0.6619047619047619 210
heroes_58_80 0.34415584415584416 154
heroes_64_66 0.6653386454183267 251
heroes_64_67 0.680365296803653 219
heroes_64_70 0.6591928251121076 223
heroes_64_93 0.651685393258427 267
heroes_89_93 0.6566265060240963 166
heroes_93_108 0.6612377850162866 307


In [20]:
train = enrich_diff(pd.read_csv('train.csv'))
train = train.merge(heroes[['mid'] + interesting_pairs], on='mid', how='left')

In [21]:
def normalize(pair_value):
    if pair_value == 1 or -1:
        return 0
    
    if pair_value == 2:
        return 1
    
    if pair_value == -2:
        return -1
    
for e in interesting_pairs:
    train[e] = train[e].map(normalize)